In [1]:
import os
import sys
sys.path.append(os.path.join(os.getcwd().replace("model_inference", "")))

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.profiler import profiler
import matplotlib.pyplot as plt
import time
from parse_dataset import NetworkDataset, parse_dataset, split_datasets, binary_dataset
from split_model import SplitModelDPU, SplitModelHost
from load_models import models
from transfer_tensors import DPUSocket

In [2]:
conf = {
    "batch_size": 512,
    "epochs": 10,
    "learning_rate": 0.0001,
    "dpu": True
}

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [4]:
# extract dataset from csv file
network_data_file = os.path.join(os.getcwd().replace("model_inference", ""), "datasets", "network_packet_data_test.csv")

data, labels, label_dict = parse_dataset(network_data_file)

X_train, y_train, X_val, y_val, X_test, y_test = split_datasets(data, labels)

X_train, X_val, X_test = X_train.unsqueeze(-1), X_val.unsqueeze(-1), X_test.unsqueeze(-1)
print(X_train.shape)
print(y_train.shape)

# create train, val and test datasets
train_dataset = NetworkDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=conf["batch_size"], shuffle=True)

val_dataset = NetworkDataset(X_val, y_val)
val_loader = DataLoader(val_dataset, batch_size=conf["batch_size"], shuffle=True)

test_dataset = NetworkDataset(X_test, y_test)
test_loader = DataLoader(test_dataset, batch_size=conf["batch_size"])

packet_dat,attack_cat

torch.Size([356334, 513, 1])
torch.Size([356334])


In [5]:
dpu_path = os.path.join(os.getcwd().replace("model_inference", ""), "checkpoint", "dpu_split_model.pth")
dpu_model = models["dpu"]
dpu_model.load(dpu_path)

Checkpoint loaded from /home/jorgetf/testmodel/Network-Packet-ML-Model/checkpoint/dpu_split_model.pth!


In [8]:
so_file = os.path.join(os.getcwd().replace("model_inference", ""), "socket_transfer", "socket_transfer.so")
address = "127.0.0.1"
socket = DPUSocket(so_file, address)

In [6]:
# test inference:
data, labels = next(iter(test_loader))
if not data.is_cuda or not labels.is_cuda:
    data, labels = data.to(device), labels.to(device)

In [7]:
dpu_model.model.eval()
pred, logits, targets = dpu_model.inference(data, labels)
print(pred.shape, logits.shape, targets.shape)
print(targets[:10])
print(pred[:10])

torch.Size([512]) torch.Size([430, 1, 128]) torch.Size([430])
tensor([ 0,  0, 14,  0, 12, 10, 15, 10, 19, 14])
tensor([1., 1., 1., 1., 1., 1., 0., 1., 1., 0.])


In [10]:
targets = targets.to(dtype=torch.float)
print(targets[:10])

tensor([ 0.,  0., 14.,  0., 12., 10., 15., 10., 19., 14.])


In [11]:
# send logits
socket.send(logits)

Buffer of size 220160 bytes sent to host! 


In [12]:
#send targets
socket.send(targets)

Buffer of size 1720 bytes sent to host! 
